In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasters as rt
from rasters import MultiPoint
from PTJPLSM import PTJPLSM, process_PTJPLSM_table
from PTJPL import load_Topt, load_fAPARmax
from soil_capacity_wilting import load_field_capacity
from soil_capacity_wilting import load_wilting_point
from gedi_canopy_height import load_canopy_height
from ECOv002_calval_tables import load_combined_eco_flux_ec_filtered, load_metadata_ebc_filt, load_calval_table

In [2]:
repo_root = os.path.dirname(os.getcwd())
package_dir = os.path.join(repo_root, 'PTJPLSM')
generated_input_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-SM-inputs.csv")
generated_output_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-SM-outputs.csv")

In [3]:
model_inputs_gdf = load_calval_table()
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,MAP,StartDate,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1320.00,10/1/18 05:00,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681


In [4]:
# extract array of x and array of y coordinates from data frame geometry
x_coords = model_inputs_gdf.geometry.x
y_coords = model_inputs_gdf.geometry.y
tower_points = MultiPoint(x=x_coords, y=y_coords)
tower_points

MULTIPOINT ((-76.656 35.799), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-75.9038 35.7879), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-90.3004 45.9793), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6

In [5]:
Topt = load_Topt(geometry=tower_points)
model_inputs_gdf["Topt"] = Topt
Topt

array([10.09,  1.39,  1.39, ...,  7.99,  7.99,  7.99], shape=(1065,))

In [6]:
fAPARmax = load_fAPARmax(geometry=tower_points)
model_inputs_gdf["fAPARmax"] = fAPARmax
fAPARmax

array([0.4659, 0.4865, 0.4865, ..., 0.3387, 0.3387, 0.3387], shape=(1065,))

In [7]:
canopy_height_meters = load_canopy_height(geometry=tower_points)
model_inputs_gdf["canopy_height_meters"] = canopy_height_meters
canopy_height_meters

array([20.64290228,  0.        ,  0.        , ...,  0.        ,
        0.        ,  0.        ], shape=(1065,))

In [8]:
field_capacity = load_field_capacity(geometry=tower_points)
model_inputs_gdf["field_capacity"] = field_capacity
field_capacity

array([0.24, 0.31, 0.31, ..., 0.28, 0.28, 0.28], shape=(1065,))

In [9]:
wilting_point = load_wilting_point(geometry=tower_points)
model_inputs_gdf["wilting_point"] = wilting_point
wilting_point

array([0.11, 0.14, 0.14, ..., 0.13, 0.13, 0.13], shape=(1065,))

In [10]:
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,geometry,time_UTC,ST_K,ST_C,Ta_C,Topt,fAPARmax,canopy_height_meters,field_capacity,wilting_point
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920,10.09,0.4659,20.642902,0.24,0.11
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982,1.39,0.4865,0.000000,0.31,0.14
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862,1.39,0.4865,0.000000,0.31,0.14
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096,1.39,0.4865,0.000000,0.31,0.14
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691,1.39,0.4865,0.000000,0.31,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752,7.99,0.3387,0.000000,0.28,0.13
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186,7.99,0.3387,0.000000,0.28,0.13
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336,7.99,0.3387,0.000000,0.28,0.13
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681,7.99,0.3387,0.000000,0.28,0.13


In [11]:
model_inputs_gdf.keys()

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time', 'Site ID', 'Name', 'Lat',
       'Long', 'Elev', 'Clim', 'Veg', 'MAT', 'MAP', 'StartDate', 'EndDate',
       'LE_count', 'closure_ratio', 'geometry', 'time_UTC', 'ST_K', 'ST_C',
       'Ta_C', 'Topt', 'fAPARmax', 'canopy_height_meters', 'field_capacity',
       'wiltin

In [12]:
model_inputs_gdf.albedo

0       0.215445
1       0.117238
2       0.117280
3       0.084629
4       0.120526
          ...   
1060    0.092853
1061    0.111844
1062    0.106782
1063    0.106775
1064    0.113165
Name: albedo, Length: 1065, dtype: float64

In [13]:
results = process_PTJPLSM_table(model_inputs_gdf)
results

[2025-08-13 11:09:26 INFO] variable fAPARmax min: 0.233 mean: 0.382 max: 0.624 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable Ta_C min: -14.605 mean: 22.322 max: 39.710 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable RH min: 0.273 mean: 0.427 max: 0.984 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable soil_moisture min: 0.000 mean: 0.168 max: 0.897 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable field_capacity min: 0.190 mean: 0.283 max: 0.390 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable wilting_point min: 0.040 mean: 0.121 max: 0.290 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable canopy_height_meters min: 0.000 mean: 6.044 max: 30.192 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable Rn_Wm2 min: 0.000 mean: 414.279 max: 763.136 nan: 0.00% (nan)
[2025-08-13 11:09:26 INFO] variable G min: 0.000 mean: 51.435 max: 158.322 nan: 0.19% (nan)
[2025-08-13 11:09:26 INFO] variable fwet min: 0.000 mean: 0.015 max: 0.936 nan: 0.00% (nan)
[2025-08-13 11:09:26 IN

/Users/halverso/Projects/PT-JPL-SM/PTJPLSM/model.py:409: RuntimeWarning: invalid value encountered in divide
  EF = rt.where((LE_Wm2 == 0) | ((Rn_Wm2 - G_Wm2) == 0), 0, LE_Wm2 / (Rn_Wm2 - G_Wm2))


,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,LE_soil_Wm2,Rn_canopy_Wm2,PET_Wm2,LE_canopy_Wm2,LE_interception_Wm2,LE_Wm2,Rn_daily_Wm2,EF,LE_daylight_Wm2,ET_daily_kg
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,57.976354,285.831187,348.579227,249.754179,0.029060,307.759593,235.472599,0.897636,211.368743,3.629675
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,149.111715,400.491330,518.484847,233.201800,0.036937,382.350453,333.755877,0.680139,227.000410,5.018461
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,165.019855,373.689380,518.782458,109.059177,0.035376,274.114408,324.540209,0.500205,162.336529,3.585863
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,124.726104,423.480937,505.753641,117.661922,0.038103,242.426129,339.074340,0.431284,146.237189,3.225827
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,94.663734,350.798478,413.852320,132.090657,0.031919,226.786310,300.989194,0.498610,150.076259,3.308545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,0.006812,41.796593,92.388700,14.085992,0.002426,14.095231,124.535624,0.088569,11.029977,0.156662
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,0.009935,34.292221,128.608037,20.265869,0.002926,20.278730,177.871717,0.134519,23.927204,0.426937
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,0.014220,64.307806,207.238994,54.596389,0.006504,54.617113,241.740261,0.266548,64.435363,1.212173
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,0.011312,67.875998,161.062364,39.968092,0.004795,39.984198,213.453822,0.175358,37.430729,0.708064


In [14]:
model_inputs_gdf.to_csv(generated_input_table_filename, index=False)

In [15]:
results.to_csv(generated_output_table_filename, index=False)